In [10]:
# Get top 50 movies of genre of your choice and along with the choices of torrent links for them

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import io
import re

In [2]:
def make_request(URL):
    output = requests.get(url = URL)
    output = output.content
    return BeautifulSoup(output,'html5lib')

In [11]:
search_case = ['Action','Adventure','Animation','Biography','Comedy','Crime','Documentary','Drama','Family','Fantasy','History','Horror','Music','Musical','Mystery','Romance','Sci-Fi','Short','Sport','Superhero'
,'Thriller','War','Western']
list_genre = {}
i = 0
for val in search_case:
    list_genre[i] = val
    i += 1
search_case = int(input(f" Get list of top 50 movies of Genre - {list_genre}"))

 Get list of top 50 movies of Genre - {0: 'Action', 1: 'Adventure', 2: 'Animation', 3: 'Biography', 4: 'Comedy', 5: 'Crime', 6: 'Documentary', 7: 'Drama', 8: 'Family', 9: 'Fantasy', 10: 'History', 11: 'Horror', 12: 'Music', 13: 'Musical', 14: 'Mystery', 15: 'Romance', 16: 'Sci-Fi', 17: 'Short', 18: 'Sport', 19: 'Superhero', 20: 'Thriller', 21: 'War', 22: 'Western'}20


In [13]:
URL = 'https://www.imdb.com/search/title/?genres='+list_genre[search_case]+'&num_votes=1000,&sort=user_rating,desc&title_type=feature'
soup1 = make_request(URL)
titles = soup1.find_all('div',class_='lister-item mode-advanced')
movie_table = {}

# movie list ka dictionary create ho gaya hai ab yaha pe
for data in titles:
    lis = data.find_all('div',class_="inline-block ratings-imdb-rating")
    movie_table[data.h3.a.get_text()] = lis[0].find_all('strong')[0].get_text()

In [14]:
main_page_start = "<html><head><title> Movie and torrents </title> </head> <body> <table border='1'>"
main_page_end = "</table></body></html>"
main_page_middle = ""
for keys,values in movie_table.items():
    link_1 = "https://www.1377x.to/search/"+ keys +"/1/"
    soup2 = make_request(link_1)
    value_to_be_added_html = None
    if soup2.find_all('div',class_="box-info-heading clearfix"):
        tables = soup2.find_all('tr') # get all the rows having links in the table
        tables.pop(0) # remove the header row that is unnecessary
        html_start_string,html_end_string,html_mid_string = "<table>","</table>",""
        link_2,link_3 = None,None
        for data in tables:
            temp1 = data.find_all('a')[1]
            link_2 = "https://www.1377x.to" + temp1['href']
            soup3 = make_request(link_1)
            temp2 = soup3.find_all('a' , href = re.compile("(^magnet+)"))
            if temp2:
                link_3 = temp2[0]['href']
            else:
                link_3 = link_2
            html_mid_string += "<tr><td>"+ temp1.get_text() +"</td><td><a href = "+ link_3 + "> " +link_2+"</a></td></tr>"
        value_to_be_added_html = html_start_string + html_mid_string + html_end_string
    else:
        value_to_be_added_html = "<h3> No Torrents Found for this movie. </h3>"
    main_page_middle += "<tr><td>"+ keys +"</td><td>"+ values +"</td><td>"+ value_to_be_added_html +"</td></tr>"
with io.open('links.html', "w", encoding="utf-8") as f:
    f.write(main_page_start + main_page_middle + main_page_end)
f.close()